In [11]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [12]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [13]:
#フォルダutilsにあるdataloader_image_classification.pyをインポート
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

#アリとハチの画像のパスを格納したリストを作成
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

#Datasetを作成
size = 224
mean = (0.485, 0.456, 0.406) #ImageNetのデータの平均値
std = (0.229, 0.224, 0.225) #ImageNetのデータの標準偏差
train_dataset = HymenopteraDataset(file_list = train_list, transform = ImageTransform(size, mean, std), phase="train")
val_dataset = HymenopteraDataset(file_list = val_list, transform = ImageTransform(size, mean, std), phase="val")

#DataLoaderを作成
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#辞書型オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [14]:
#学習済みのVGG16モデルをロード

#VGG16のインスタンスを作成
use_pretrained = True #学習済みのパラメータを使用
net = models.vgg16(pretrained = use_pretrained)

#VGG16の最後の出力層の出力ユニット数をアリとハチの2つに変更
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

#訓練モードに設定
net.train()

print("ネットワーク設定完了：学習済みの重みをロードし、訓練モードに変更しました。")



/opt/conda/envs/py38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/py38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ネットワーク設定完了：学習済みの重みをロードし、訓練モードに変更しました。


In [16]:
#損失関数の設定
criterion = nn.CrossEntropyLoss()

In [18]:
#VGG16のモデル構成を表示
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
#ファインチューニングで学習させるパラメーターを、変数params_to_updateの1~3に格納する
